In [1]:
import torch

In [2]:
torch.__version__

'1.12.0.dev20220408+cu113'

In [3]:
model_name = "t5-large"

In [4]:
from transformers import AutoTokenizer, AutoModelWithLMHead,AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelWithLMHead.from_pretrained(model_name)


/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:907: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [5]:
model.load_state_dict(torch.load("t5large_2e.pt")) #("t5-smalltrain_acc4.pt"))


<All keys matched successfully>

In [6]:
model.eval();

In [7]:
import datasets_grammar as dg
from defaults import train_config
import os

In [8]:
cfg = train_config()

In [9]:
test_dataset = dg.get_dataset(tokenizer, cfg.dataset_test, 512, 512, True)
print(len(test_dataset))
print(f"using dataset {cfg.dataset_test}")

Using custom data configuration default-e2d071caf158a2c7
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-e2d071caf158a2c7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

2988
using dataset datasets_grammar/grammar_validation.csv


In [22]:
def validate_one_step(model, batch):
    #print(batch.keys())
    curr_loss = torch.zeros(3)#.to("cuda:0")
    for key in batch.keys():
        #batch[key] = batch[key].to("cuda:0")
        output = model(
                input_ids=batch["source_ids"],
                attention_mask=batch["source_mask"],
                labels=batch["target_ids"],
            )
        curr_loss[0] += output["loss"].item()  # sum up batch loss
        curr_loss[1] +=len(batch)
        #print(curr_loss)
        #pred = output.logits.argmax(
         #       dim=1, keepdim=True
         #   )  # get the index of the max log-probability
        #print(pred)
        #curr_loss[1] += pred.eq(batch["target_ids"].view_as(pred)).sum().item()
        #curr_loss[2] += len(batch)
        #print(output)
        #loss = 0
    #loss = model(**data)
    return curr_loss


In [28]:
def validate_one_epoch(model, data_loader):
    model.eval()
    total_loss = torch.zeros(3)
    for batch_index, data in enumerate(data_loader):
        with torch.no_grad():
            loss = validate_one_step(model, data)
        total_loss += loss
        
        print(total_loss[0]/total_loss[1])
        if batch_index > 4:
            break
    return total_loss

In [29]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [30]:
#model.to("cuda:0");

In [31]:
test_loss = validate_one_epoch(model,test_dataloader)

tensor(0.0040)
tensor(0.0031)
tensor(0.0027)
tensor(0.0046)
tensor(0.0042)
tensor(0.0040)


In [32]:
test_loss

tensor([ 0.3809, 96.0000,  0.0000])

In [33]:
test_loss[0]/test_loss[1]

tensor(0.0040)

In [55]:
test = "The cherry blossoms was so pretty"
test2 = "Our dogs is running in the park."
test3 = "Their the ones who made a mistake."
test4 = "Its cold outside today."
test5 = "The baby was held by its mother."
test6 = "The book on AI really effected me."
test7= "The children love eating, coloring, and to play with their toys."

In [56]:
text = test7 #"I is reading about AI articles "
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')

output = model.generate(inputs['input_ids'], num_beams=5, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
result=("".join(correction))


In [57]:
print(f"input: {text}\n")
print(f"corrected: {result}")

input: The children love eating, coloring, and to play with their toys.

corrected: The children love eating, coloring, and playing with their toys.
